In [1]:
username = "Alex"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
include(pathtorepo * "dynamical-systems\\system.jl")


using StaticArrays, DifferentialEquations, DynamicalSystems, JLD
E, x, y  = -40..40, -5..5,  -5..5
box = E × x × y
using GLMakie
using LinearAlgebra, LinearSolve,Statistics

cd("C:\\Users\\" *username *  "\\Desktop\\dynamical-systems\\Tsodyks Markram\\Levanova\\3 набор параметров\\Сопоставление с матконт\\файлы matlab")
I0_hom = load("I0_hom_hom.jld")["data"]
u0_hom = load("U0_hom_hom.jld")["data"]
I0_hom = I0_hom[:]
u0_hom = u0_hom[:];

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`


In [2]:
function takeevent(fp, ϵ_box)
function condition(u, t, integrator)
        x = Vector(u)
        b = x - fp[1]
        b = Vector(b)
        linprob = LinearProblem(A, b)
        linsolve = solve(linprob)
        return norm(linsolve.u, Inf) - ϵ_box
    end
    return condition
end
affect!(integrator) = terminate!(integrator)
function get_norm_αs(u)
    x = Vector(u)
    b = x - fp[1]
    b = Vector(b)
    linprob = LinearProblem(A, b)
    linsolve = solve(linprob)
    return norm(linsolve.u, Inf), linsolve
end
function color_alpha(alpha)
    if alpha > 0.0
        return "red"
    else
        return "blue"
    end
end;

In [3]:
const τ = 0.013;  const τD = 0.07993;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4; const α = 1.58; const ΔU0 = 0.305
t = 1000.0; tspan = [0.0, t];

In [4]:
# index point from homoclinic curve
index_hom = 1
I0 = I0_hom[index_hom]; U0 = u0_hom[index_hom]

p = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0 = [0.0, 0.0, 0.0]

ds = CoupledODEs(TM, u0, p)
fp, _, _ = fixedpoints(ds, box, jacob_TM_)

ϵ_box = 1e-6
Ju0 = jacob_TM_(fp[1], p, 0);
fp_ = fp[1]

eigen_val_vec = eigen(Ju0)
eigen_vectors = eigen_val_vec.vectors

v1 = real(eigen_vectors[:, 1])
v2 = real(eigen_vectors[:, 2])
v3 = imag(eigen_vectors[:, 3])

v1 = normalize(v1, 2)
v1t = transpose(v1)

v2 = normalize(v2, 2)
v2t = transpose(v2)

v3 = normalize(v3, 2)
v3t = transpose(v3)

A = transpose([v1t; v2t; v3t])
A = Matrix(A)

3×3 Matrix{Float64}:
  0.998728    0.999899     0.0
 -0.0489879  -0.0141814    0.984697
  0.0119121   0.000705852  0.174274

In [5]:
total_number_points = 25
number_points_on_side = Int((total_number_points) / 1)

dots_right_side = zeros(number_points_on_side, length(u0))
array_α_vec_right = zeros(number_points_on_side, length(u0))

dots_left_side = zeros(number_points_on_side, length(u0))
array_α_vec_left = zeros(number_points_on_side, length(u0))

dots_up_side = zeros(number_points_on_side, length(u0))
array_α_vec_up = zeros(number_points_on_side, length(u0))

dots_down_side = zeros(number_points_on_side, length(u0))
array_α_vec_down = zeros(number_points_on_side, length(u0));

In [6]:
# case α1 = 0; α2 = ϵ_box; α3 = [-ϵ_box; ϵ_box]
n = 1
N = total_number_points
while n <= number_points_on_side

    α1 = 0.0
    α2 = ϵ_box
    α3 = - ϵ_box + 2 * ϵ_box * n / N
    α_vec = [α1, α2, α3]
    point = fp_ + A * α_vec
    
    dots_right_side[n, :] = point
    _, α_vec = get_norm_αs( dots_right_side[n, :] )
    array_α_vec_right[n, :] = α_vec
    n+=1
end

# case α1 = 0; α2 = -ϵ_box; α3 = [-ϵ_box; ϵ_box]
n = 1
N = total_number_points
while n <= number_points_on_side

    α1 = 0.0
    α2 = -ϵ_box
    α3 = - ϵ_box + 2 * ϵ_box * n / N
    α_vec = [α1, α2, α3]
    point = fp_ + A * α_vec
    
    dots_left_side[n, :] = point
    _, α_vec = get_norm_αs( dots_left_side[n, :] )
    array_α_vec_left[n, :] = α_vec
    n+=1
end

# case α1 = 0; α2 = [-ϵ_box; ϵ_box]; α3 = ϵ_box
n = 1
N = total_number_points
while n <= number_points_on_side

    α1 = 0.0
    α2 = - ϵ_box + 2 * ϵ_box * n / N
    α3 = ϵ_box
    α_vec = [α1, α2, α3]
    point = fp_ + A * α_vec
    
    dots_up_side[n, :] = point
    _, α_vec = get_norm_αs( dots_up_side[n, :] )
    array_α_vec_up[n, :] = α_vec
    n+=1
end

# case α1 = 0; α2 = [-ϵ_box; ϵ_box]; α3 = -ϵ_box
n = 1
N = total_number_points
while n <= number_points_on_side

    α1 = 0.0
    α2 = - ϵ_box + 2 * ϵ_box * n / N
    α3 = -ϵ_box
    α_vec = [α1, α2, α3]
    point = fp_ + A * α_vec
    
    dots_down_side[n, :] = point
    _, α_vec = get_norm_αs( dots_down_side[n, :] )
    array_α_vec_down[n, :] = α_vec
    n+=1
end;

In [7]:
Es = Float64[]; xs = Float64[]; ys = Float64[]
α1s =  Float64[]; α2s =  Float64[]; α3s =  Float64[]

colors = []
check_events = []
time_events = []

Any[]

In [8]:
condition = takeevent(fp, ϵ_box)
index = 3
cb = ContinuousCallback(condition, nothing, affect!)
u0 = SA[dots_down_side[index, 1], dots_down_side[index, 2], dots_down_side[index, 3]]
prob = ODEProblem(TM, u0, tspan, p)
sol = solve(prob, alg = Vern9(), abstol = 1e-14, reltol = 1e-14, callback = cb)

retcode: Success
Interpolation: specialized 9th order lazy interpolation
t: 95511-element Vector{Float64}:
    0.0
    0.03556498573021014
    0.05831145724980826
    0.0821684611389108
    0.10871519138867278
    0.13485482970742096
    0.16167542379245267
    0.18718805212602788
    0.21157140963749077
    0.23590547113586668
    ⋮
  999.9219931755918
  999.9316376740837
  999.94127210547
  999.9519239110459
  999.9615559698381
  999.9707059309701
  999.9806585225937
  999.9910194209779
 1000.0
u: 95511-element Vector{SVector{3, Float64}}:
 [8.345807691697402, 0.7384936907334415, 0.4382983855424371]
 [8.345754022995926, 0.7384945019223078, 0.43829835650793997]
 [8.345721583154427, 0.7384951270487772, 0.4382983628165491]
 [8.345693807719647, 0.7384957989358722, 0.4382983924022813]
 [8.345675077519651, 0.7384964749013222, 0.43829845180041227]
 [8.345673116322027, 0.7384969672799124, 0.4382985326373106]
 [8.34569045070977, 0.7384971999122557, 0.43829862955926635]
 [8.345725021306855, 0.

In [9]:
norm_, linsolve = get_norm_αs(sol[end])

(3.5666214225922928, [0.47281146428392057, -3.5666214225922928, -0.0035863979642904846])

In [10]:
# last points in space (α2, α3)
f = Figure(resolution = (700, 700))

ax = Axis(f[1, 1], xlabel = L"α_2", ylabel = L"α_3", xlabelsize = 35, ylabelsize = 35)
idx, idy = 5, 6

scatter!(ax, 0.0, 0.0, color = :green, markersize = 15, marker = :rect)

scatter!(ax, linsolve[2], linsolve[3], color = :black, markersize = 5)


display(GLMakie.Screen(), f);

In [ ]:
# last points in phase space
idx, idy, idz = 2, 3, 1
tstart, tend = 1, length(sol)
f = Figure(resolution = (700, 700))

ax = LScene(f[1, 1], show_axis = true)
    
scale!(ax.scene, 50, 50, 1)
lines!(ax, sol[idx, tstart:tend], sol[idy, tstart:tend], sol[idz, tstart:tend], linewidth = 1.0, color = :deeppink)
scatter!(fp[1][idx], fp[1][idy], fp[1][idz])

display(GLMakie.Screen(), f);